[data](https://o365coloradoedu-my.sharepoint.com/:f:/g/personal/anza4273_colorado_edu/Emctie6mJf5PsXxOzXd_jawBO2UcDfpsXFRFlvxeAXSmxA?e=abpIHF)

In [6]:
import pickle

import bag_utils as utils

%matplotlib inline


tmp_file = 'tmp_data.pkl'

In [3]:
import csv
import datetime
import os

import numpy as np
import ipywidgets as widgets
import matplotlib.animation as anim
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm

import rosbag
from sensor_msgs.msg import Imu
from sensor_msgs import point_cloud2 as pc2
import tf.transformations as tf

#### 0. Read and process the data, takes a while, processed stats can be saved in a file.

In [3]:
data = utils.process_bags(bags_folder='deployment_nov2023', return_dates=True, show_plots=False)
with open(tmp_file, 'wb') as f:
    pickle.dump(data, f)

Processing 114 files


  0%|                                                   | 0/114 [00:00<?, ?it/s]/home/arpg/RWM/bag_utils.py:231: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_max_intens_mode = stats.mode(y_max_intens.round(2)).mode[0]
/home/arpg/RWM/bag_utils.py:232: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to Tru

#### 1. Load preprocessed data

In [3]:
with open(tmp_file, 'rb') as f:
    data = pickle.load(f) 
gt = utils.read_gt('deployment_nov2023/GT_Nov2023.csv')

#### 2. Plot data

In [5]:
utils.plot_total(data, gt, label_dates=True)

TypeError: unsupported operand type(s) for -: 'NoneType' and 'NoneType'